In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers

In [ ]:
import torch

if torch.cuda.is_available:
    print("Cuda is available")
    device = torch.device('cuda')
else:
    print("Cuda not available. Use CPU")
    device = torch.device('cpu')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_en2vi = AutoTokenizer.from_pretrained("vinai/vinai-translate-en2vi", src_lang="en_XX")
model_en2vi = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-en2vi")
model_en2vi.to(device)

model_en2vi.eval()

In [ ]:
def translate_en2vi(en_texts: str) -> str:
    input_ids = tokenizer_en2vi(en_texts, padding=True, return_tensors="pt").to(device)
    output_ids = model_en2vi.generate(
        **input_ids,
        decoder_start_token_id=tokenizer_en2vi.lang_code_to_id["vi_VN"],
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )
    vi_texts = tokenizer_en2vi.batch_decode(output_ids, skip_special_tokens=True)
    return vi_texts

In [ ]:
# The input may consist of multiple text sequences, with the number of text sequences in the input ranging from 1 up to 8, 16, 32, or even higher, depending on the GPU memory.
en_texts = ["After realizing that money means nothing when his mother is depressed and alone by herself, Phan Van Chieu decided to bring her back to their hometown and flip over a new page as a content creator.",
            "On an early morning in late September, Nguyen Thi Trinh, 45, was hard at work making jelly mooncakes with salted egg yolk filling in at small Mekong Delta home in Binh Chanh 1 Village, Binh My Town, An Giang Province. She worked late into the night, satisfied with her batch.",
            "The whole day and night of work was neatly captured in a two-minute video shared by her son Chieu on TikTok just in time for the Mid-Autumn Festival. Every few minutes, Trinh would open her phone to see the likes and read the comments, a smile brightening her face. Creating content was a joy."]

print(translate_en2vi(en_texts))